In [2]:
import pyodbc
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as FF
import datetime as dt
import json
from scipy import stats
import numpy as np
import sys
import math

In [5]:
dfHDI_raw = pd.read_csv('HDI_MPI_GDI_EF_Merged_v001.csv', encoding = "ISO-8859-1")
dfNFA_raw = pd.read_csv('NFA 2018 Edition.csv')
dfCPI_raw = pd.read_csv('CPI 2015_data.csv')
dfGINI = pd.read_csv('GINI.csv')
dfGDP = pd.read_csv('GDP.csv')
dfNFA_raw = dfNFA_raw.loc[dfNFA_raw['year'] == 2014]
dfNFA_raw = dfNFA_raw[dfNFA_raw['record'].isin(['EFProdTotGHA' , 'EFConsTotGHA'])]
del dfNFA_raw['year']
del dfNFA_raw['country_code']
del dfNFA_raw['record']
del dfNFA_raw['QScore']
dfNFA_raw = dfNFA_raw.rename(columns={'country': 'Country'})
df_grouped = dfNFA_raw.groupby(['Country'], as_index=False).aggregate(np.sum)

columns_CPI = [ 'CPI2015', 'Country', 'World Bank CPIA',
       'World Economic Forum EOS', 'Bertelsmann Foundation TI']

columns_HDI = ['Country', 
       'Human_Development_Index', 'Life_expectancy_at_birth',
       'Expected_years_of_schooling', 'Mean_years_of_schooling',
       'Gross_national_income_GNI_per_capita']
dfHDI = dfHDI_raw [ columns_HDI ]
dfCPI = dfCPI_raw [ columns_CPI]
df1 = pd.merge(dfHDI, dfCPI, how='inner', on=['Country'])
df_Merged = pd.merge(df_grouped, df1, how='inner', on=['Country'])

GINI_list = dfGINI['Country'].tolist()

GINI_new_countries = []
for i in GINI_list:
    country = i[1:]
    GINI_new_countries.append(country)
    
new_country = pd.Series(GINI_new_countries)
dfGINI['Country'] = new_country.values
df_Merged = pd.merge(dfGINI, df_Merged, how='inner', on=['Country'])
df_Merged = pd.merge(dfGDP, df_Merged, how='inner', on=['Country'])

del df_Merged['World Bank CPIA']
del df_Merged['World Economic Forum EOS']
del df_Merged['Bertelsmann Foundation TI']

df_Merged = df_Merged.rename(columns={'total': 'EFtotal', '2015': 'GDP'})
df_merged = df_Merged.dropna(how='any')
df_merged.to_csv("pre_pro_EFtot.csv")